# Trainer

Using the DEiT small model here (DEiT Small Distilled Patch 16, Image size 244 x 244) in the interest of time and space for deployment

In [1]:
import bentoml

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from keras.applications import imagenet_utils

from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-12-19 03:15:29.854552: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 03:15:30.769921: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11/lib64/
2022-12-19 03:15:30.772163: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11/lib64/
2022-12-19 03:15:30.772195: W tensorflow/compiler/tf2tensorrt/utils/py_utils

In [2]:
DATASET_SIZE = 9367
IMAGE_SIZE = 224
BATCH_SIZE = 8
WORKERS = 4
EPOCHS = 10

BASE_PATH='./data'

classes = [
    'cup', 
    'fork', 
    'glass', 
    'knife', 
    'plate', 
    'spoon'
]

First, we will load the training dataframe and split it into train and validation

In [3]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,data/images/0560.jpg
1,4675,cup,data/images/4675.jpg
2,0875,glass,data/images/0875.jpg
3,4436,spoon,data/images/4436.jpg
4,8265,plate,data/images/8265.jpg


In [4]:
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

## Training

Now let's create image generators

In [5]:
# These models don't have the imagenet preprocessing built in so I have to apply this
def preprocess_input(x, data_format=None):
    return imagenet_utils.preprocess_input(
        x, data_format=data_format, mode="tf"
    )

In [6]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    dtype="float16"
)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    dtype="float16"
)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 4447 validated image filenames belonging to 6 classes.
Found 1112 validated image filenames belonging to 6 classes.


In [7]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [8]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    min_delta = 1e-4,
    patience = 3,
    mode = 'max',
    restore_best_weights = True,
    verbose = 1
)

checkpoint = keras.callbacks.ModelCheckpoint(
    'deit_s_d_p16_224_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max'
)

callbacks = [earlystopping, checkpoint]

In [9]:
def get_model_deit(model_url, res=IMAGE_SIZE, num_classes=len(classes)) -> tf.keras.Model:
    inputs = tf.keras.Input((res, res, 3))
    hub_module = hub.KerasLayer(model_url, trainable=False)

    base_model_layers, _ = hub_module(inputs)   # Second output in the tuple is a dictionary containing attention scores.
    outputs = keras.layers.Dense(num_classes, activation="softmax")(base_model_layers)
    
    return tf.keras.Model(inputs, outputs) 

Warnings are normal; the pre-trained weights for the original classifications heads are being skipped.

In [10]:
def build_model():
    model_gcs_path = "http://tfhub.dev/sayakpaul/deit_base_distilled_patch16_224_fe/1"
    model = get_model_deit(model_gcs_path)

    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = 1e-2
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [11]:
model = build_model()

2022-12-19 03:15:41.237133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-19 03:15:41.256940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-19 03:15:41.257192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-19 03:15:41.257554: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [12]:
history = model.fit(
    x = train_generator,
    validation_data=val_generator,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    workers=WORKERS,
    callbacks=callbacks
)

Epoch 1/10


2022-12-19 03:15:58.733548: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2022-12-19 03:16:01.021057: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-19 03:16:04.280256: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-12-19 03:16:04.531486: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x556dffb7d350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-19 03:16:04.531555: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2022-12-19 03:16:04.568026: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-12-19 03:16:05.00597

556/556 [==============================] - 93s 140ms/step - loss: 0.2853 - accuracy: 0.9395 - val_loss: 0.2532 - val_accuracy: 0.9550
Epoch 2/10
556/556 [==============================] - 71s 127ms/step - loss: 0.1513 - accuracy: 0.9683 - val_loss: 0.3108 - val_accuracy: 0.9586
Epoch 3/10
556/556 [==============================] - 67s 120ms/step - loss: 0.1619 - accuracy: 0.9730 - val_loss: 0.4528 - val_accuracy: 0.9442
Epoch 4/10
556/556 [==============================] - 69s 125ms/step - loss: 0.0913 - accuracy: 0.9813 - val_loss: 0.3287 - val_accuracy: 0.9559
Epoch 5/10
556/556 [==============================] - 75s 135ms/step - loss: 0.0937 - accuracy: 0.9849 - val_loss: 0.4594 - val_accuracy: 0.9541
Epoch 5: early stopping


In [19]:
image_path = 'testing/0966.jpg'

image = tf.keras.utils.load_img(
    image_path,
    target_size=(IMAGE_SIZE, IMAGE_SIZE)
)
input_arr = tf.keras.utils.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.
predictions = model.predict(input_arr)

1/1 [==============================] - 5s 5s/step


In [22]:
print(predictions)
predicted_class = classes[np.argmax(predictions, axis=1)[0]]
predicted_class

[[6.4358930e-03 1.9068438e-04 5.0141307e-04 2.8762501e-03 9.8999572e-01
  2.2158536e-08]]


'plate'

In [13]:
import bentoml
bentoml.tensorflow.save_model(
    "kitchenware-classification", 
    model,
    signatures={"__call__": {"batchable": True, "batch_dim": 0}}
)

/home/andre/anaconda3/envs/kitchenware-classification/lib/python3.9/site-packages/bentoml/_internal/frameworks/utils/tensorflow.py:305: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  function_map = {k: getattr(m, k, None) for k in dir(m)}
/home/andre/anaconda3/envs/kitchenware-classification/lib/python3.9/site-packages/bentoml/_internal/frameworks/utils/tensorflow.py:305: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  function_map = {k: getattr(m, k, None) for k in dir(m)}


INFO:tensorflow:Assets written to: /tmp/tmpxzm99qlqbentoml_model_kitchenware-classification/assets


INFO:tensorflow:Assets written to: /tmp/tmpxzm99qlqbentoml_model_kitchenware-classification/assets


Model(tag="kitchenware-classification:hglqctt7o22zyaav", path="/home/andre/bentoml/models/kitchenware-classification/hglqctt7o22zyaav/")